In [ ]:
!pip install tensorflow
!pip install keras


In [ ]:
import pandas as pd 
import os
import keras
import tensorflow as tf

from PIL import Image
from keras.layers import Conv2D, Dense, Flatten, MaxPooling2D, Input, GlobalAveragePooling2D
from keras.models import Model, Sequential
from keras.preprocessing import image
from keras.layers import Lambda
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import keras.applications.mobilenet_v2 as mobilenetv2

from google.colab import drive
drive.mount('/content/drive')

print('setup successful!')

Mounted at /content/drive
setup successful!


In [ ]:
width = 224    
height = 224
size=(width, height)
channels = 3
all_pictures = "/content/drive/My Drive/College/Colab_Notebooks/garbage_classification/"


# Dictionaries to save the 12 classes
# battery, biological,brown-glass,cardboard,clothes,green-glass,metal,paper,plastic,shoes,trash,white-glass
categories = {1: 'biological', 2: "brown-glass", 3: "cardboard", 
              4: "green-glass", 5: 'metal', 6: 'paper', 
              7: 'plastic', 8: "trash", 9: "white-glass"}
stac = {'biological': 1,"brown-glass": 2, "cardboard": 3, 
              "green-glass": 4, 'metal': 5, 'paper': 6, 
              'plastic': 7, "trash": 8, "white-glass": 9}        

defining constants successful!


In [ ]:
data = {}

for folder in os.listdir(all_pictures):
    if folder == "clothes":
      continue
    elif folder == "shoes":
      continue
    elif folder == "battery":
      continue
    # print(folder)  # This is the folder names
    for picture in os.walk(all_pictures + folder):
        # print(picture)
        # print(picture[2])
        for k in picture[2]:
            data[folder + '/' + k] = stac[folder]

data = pd.DataFrame(data.items(), columns=["filename", "category"])
df = data.sample(frac=1).reset_index(drop=True)

print('number of elements = ' , len(df))
df.head()

number of elements =  7268


,filename,category
0,biological/biological633.jpg,1
1,biological/biological823.jpg,1
2,plastic/plastic817.jpg,7
3,white-glass/white-glass71.jpg,9
4,green-glass/green-glass197.jpg,4


In [ ]:
mobilenetv2_layer = mobilenetv2.MobileNetV2(include_top = False, input_shape = (width, height,channels))
mobilenetv2_layer.trainable = False


model = Sequential()
model.add(keras.Input(shape=(width, height, channels)))

#create a custom layer to apply the preprocessing
def mobilenetv2_preprocessing(img):
  return mobilenetv2.preprocess_input(img)

model.add(Lambda(mobilenetv2_preprocessing))

model.add(mobilenetv2_layer)
model.add(tf.keras.layers.GlobalAveragePooling2D())
model.add(Dense(len(categories), activation='softmax')) 

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])

model.summary()

9406464/9406464 [==============================] - 0s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lambda (Lambda)             (None, 224, 224, 3)       0         
                                                                 
 mobilenetv2_1.00_224 (Funct  (None, 7, 7, 1280)       2257984   
 ional)                                                          
                                                                 
 global_average_pooling2d (G  (None, 1280)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 9)                 11529     
                                                                 
Total params: 2,269,513
Trainable params: 11,529
Non-trainable params: 2,257,984
____________________________________________

In [ ]:
#Change the categories from numbers to names
df["category"] = df["category"].replace(categories) 

train, validate = train_test_split(df, test_size=0.2, random_state=37)

validate, test = train_test_split(validate, test_size=0.3, random_state=37)

train = train.reset_index(drop=True)
validate = validate.reset_index(drop=True)
test = test.reset_index(drop=True)

total_train = train.shape[0]
total_validate = validate.shape[0]

print('train size = ', total_validate , 'validate size = ', total_validate, 'test size = ', test.shape[0])


train size =  1017 validate size =  1017 test size =  437


In [ ]:
batch_size=20

train_datagen = image.ImageDataGenerator(
    
    ###  Augmentation Start  ###
    
    rotation_range=30,
    shear_range=0.1,
    zoom_range=0.3,
    horizontal_flip=True,
    vertical_flip = True,
    width_shift_range=0.2,
    height_shift_range=0.2
    
    ##  Augmentation End  ###
)

train_generator = train_datagen.flow_from_dataframe(
    train, 
    all_pictures, 
    x_col='filename',
    y_col='category',
    target_size=size,
    class_mode='categorical',
    batch_size=batch_size
)

validation_datagen = image.ImageDataGenerator()

validation_generator = validation_datagen.flow_from_dataframe(
    test, 
    all_pictures, 
    x_col='filename',
    y_col='category',
    target_size=size,
    class_mode='categorical',
    batch_size=batch_size
)

Found 5814 validated image filenames belonging to 9 classes.
Found 437 validated image filenames belonging to 9 classes.


In [ ]:
EPOCHS = 4
history = model.fit(
    train_generator, 
    epochs=EPOCHS,
    validation_data=validation_generator,
    validation_steps=total_validate//batch_size,
    steps_per_epoch=total_train//batch_size,
    #callbacks=callbacks
)

Epoch 1/7
290/290 [==============================] - ETA: 0s - loss: 0.7489 - categorical_accuracy: 0.7577

290/290 [==============================] - 1212s 4s/step - loss: 0.7489 - categorical_accuracy: 0.7577 - val_loss: 0.4637 - val_categorical_accuracy: 0.8627
Epoch 2/7
290/290 [==============================] - 324s 1s/step - loss: 0.4044 - categorical_accuracy: 0.8614
Epoch 3/7
290/290 [==============================] - 322s 1s/step - loss: 0.3604 - categorical_accuracy: 0.8797
Epoch 4/7
290/290 [==============================] - 322s 1s/step - loss: 0.3254 - categorical_accuracy: 0.8909
Epoch 5/7
290/290 [==============================] - 324s 1s/step - loss: 0.3027 - categorical_accuracy: 0.8958
Epoch 6/7
290/290 [==============================] - 326s 1s/step - loss: 0.2769 - categorical_accuracy: 0.9056
Epoch 7/7
290/290 [==============================] - 325s 1s/step - loss: 0.2681 - categorical_accuracy: 0.9082


In [ ]:
model.save_weights("/content/drive/My Drive/College/Colab_Notebooks/model12.h5")
model.save("/content/drive/My Drive/College/Colab_Notebooks/my_model.h5", save_format='tf')